In [5]:
from ortools.linear_solver import pywraplp
from random import randrange,choice
horario = pywraplp.Solver.CreateSolver('SCIP')

S,T,P,C = 5,20,10,100
disp = []
h= [0,1,1,1]
for c in range(C):
    disp.append([])
    for t in range(T):
        disp[c].append(choice(h))
#print(disp)
#A disponibilidade de cada participante, incluindo o lider. Essa disponibilidade   é um conjunto de “slots” 
#representada numa matriz booleana de acessibilidade com uma linha por cada participante 1..C e 
#uma coluna por “slot” 1..T
#disp [C][T] = []

#O conjunto de colaboradores de cada projeto, o seu líder e o número mínimo  de reuniões semanais.
#Nesse caso para um projeto de 0 - P , está associado os colaboradores 
# ((1,2,0),1,3),((4,3,2),4,3),((4,5,6,7,8,9),6,2),((1,8,9,10,11,12,13,14),11,2),((13,15,16,17,18,19),13,2)
proj = [] 
lideres = []
for p in range(P):
    colab = set()
    for col in range(14):
        colab.add(randrange(C))
    cola = list(colab)
    l = choice(cola)
    lideres.append(l)
    hora = randrange(1,5)
    proj.append((cola,l,hora))
    

print(proj)




x = {}
for s in range(S):
    x[s]= {}
    for t in range(T):
        x[s][t] = {}
        for p in range(P):
            x[s][t][p] = {}
            for c in range(C):
                x[s][t][p][c] = horario.BoolVar(f'x[{s},{t},{p},{c}]')

def X(s,t,p,c):              # abreviatura
    return x[s][t][p][c] 
#print(round(len(proj[0][0])/2))

#Adiciona condições para que se um colaborador não esteja disponivel para certo slot
#ele não posso ser alocada a esse spot
for c in range(C):
    for t in range(T):
        if disp[c][t] == 0:
            for p in range(P):
                for s in range(S):
                    horario.Add((X(s,t,p,c))== 0 )

#cada colaborador nao pode participar de um projeto que nao é seu
for c in range(C):
    for p in range(P):
        if c not in proj[p][0]:
            for s in range(S):
                for t in range(T):
                    horario.Add((X(s,t,p,c))== 0)   


            

#checa se o mesmo colaborador esta fazendo o trabalho em 2 projetos diferentes ao mesmo tempo
for c in range(C):
    for t in range(T):
        horario.Add(sum(X(s,t,p,c) for p in range(P) for s in range(S))<= 1 )

#Adiciona condições para que se o lider não esteja disponivel não aconteça o projeto   
for p in range(P):
    l = proj[p][1]
    for d in disp[l]:
        if d == 0:
            for s in range(S):
                for c in proj[p][0]:
                    horario.Add((X(s,t,p,c))== 0 )
    
# Cada sala não pode ter mais que uma reunião ao mesmo tempo
for s in range(S):
    for t in range(T):
        horario.Add(sum(X(s,t,p,c) for p in range(P) for c in lideres) <= 1)
#Adiciona condições para que pelo menos 50% dos colaboradores de um dado projeto esteja presente em um slot 
#alocado a este projeto
for p in range(P):
    for t in range(T):
        for s in range(S):
            horario.Add(sum(X(s,t,p,c) for c in proj[p][0])  >= (X(s,t,p,proj[p][1])*round(len(proj[p][0])/2)))
     
    
#Adiciona condições para que o número de reunioes por projeto seja maior que o minimo definido
for p in range(P):
    horario.Add ( proj[p][2] <= sum(X(s,t,p,lideres[p]) for s in range (S) for t in range(T)))

    
for p in range(P):
    for s in range(S):
        for t in range(T):
            horario.Minimize(sum(X(s,t,p,c) for c in range(C)))
    

    
    
    
    
h = sum(X(s,t,p,proj[p][1])for s in range(S) for t in range(T) for p in range(P))


horario.Maximize(h)


status = horario.Solve()
print(status)
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    
    #for p in range(P):
    #    for t in range(T):
    #        for s in range(S):
    #            for c in range(C):
    #                print(X(s,t,p,c).solution_value())
    for s in range(S):
        print("Sala ", s, ":")
        for t in range(T):
            for p in range(P):
                if X(s,t,p,proj[p][1]).solution_value() == 1:
                   
                    a = []
                    for c in proj[p][0] :
                        if X(s,t,p,c).solution_value() == True:
                            a.append(c)
                    print("Slot ", t ," Projeto:", p," Colaboradores:", a )


[([96, 1, 2, 69, 39, 75, 13, 14, 81, 51, 53, 26, 94, 95], 53, 2), ([64, 34, 26, 4, 69, 58, 40, 53, 90, 27, 93, 62, 63], 58, 4), ([1, 33, 7, 41, 78, 79, 15, 50, 19, 51, 26, 59, 92, 29], 79, 1), ([96, 5, 72, 40, 10, 44, 14, 16, 29, 51, 85, 53, 54, 61], 51, 2), ([69, 6, 75, 43, 77, 12, 16, 17, 52, 53, 23, 59, 93], 17, 2), ([32, 1, 40, 75, 43, 46, 78, 51, 52, 19, 22, 89, 95], 52, 4), ([7, 11, 44, 76, 46, 48, 81, 61, 20, 53, 55, 29, 95], 95, 3), ([98, 4, 37, 40, 13, 77, 82, 52, 54, 59, 28], 4, 4), ([65, 97, 98, 36, 70, 43, 75, 47, 54, 23, 89, 26, 92, 63], 26, 4), ([98, 99, 4, 36, 41, 46, 48, 16, 52, 21, 87, 59], 46, 3)]
0
Sala  0 :
Slot  0  Projeto: 1  Colaboradores: [64, 69, 58, 27, 62, 63]
Slot  1  Projeto: 2  Colaboradores: [1, 41, 78, 79, 15, 50, 19]
Slot  2  Projeto: 6  Colaboradores: [7, 76, 81, 55, 29, 95]
Slot  3  Projeto: 8  Colaboradores: [65, 97, 36, 70, 43, 26, 63]
Slot  4  Projeto: 0  Colaboradores: [96, 1, 2, 39, 81, 53, 94]
Slot  5  Projeto: 4  Colaboradores: [69, 6, 12, 16, 